<a href="https://www.kaggle.com/code/mafiosoquasar/time-series-sales-forecasting-with-pytorch?scriptVersionId=210052378" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import argparse
import os
import random,numpy,pandas
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.transforms.functional as RF
from torchvision.models.feature_extraction import create_feature_extractor
from PIL import Image
import numpy as np
import random,cv2,pandas,os,numpy
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

nz = 100
beta1 = 0.5
lr = 0.0001
batch_size=10
ngpu=1
ngf,nc = 3,3
ndf = 64

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Random Seed:  999


In [4]:
x = pandas.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv").sort_values(by=['date', 'warehouse'])

x = torch.from_numpy(numpy.nan_to_num(x[['total_orders', 'sell_price_main', 'type_0_discount', 'type_1_discount',
                                         'type_2_discount', 'type_3_discount', 'type_4_discount',  'type_5_discount', 
                                         'type_6_discount']].to_numpy().astype(numpy.float32)).reshape((13, 308263, 9)))[::100]
x.shape #4315682

class EC_NET(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rafire = nn.Sequential(       
            nn.Linear(9, 1524),
            nn.BatchNorm1d(1524),
            nn.LeakyReLU(),
            
            nn.Linear(1524, 824),
            nn.BatchNorm1d(824),
            nn.LeakyReLU(),
            
            nn.Linear(824, 424),
            nn.BatchNorm1d(424),
            nn.LeakyReLU(),
            
            nn.Linear(424, 824),
            nn.BatchNorm1d(824),
            nn.LeakyReLU(),
            
            nn.Linear(824, 1524),
            nn.BatchNorm1d(1524),
            nn.LeakyReLU(),

            nn.Linear(1524, 10)
            
        )

    def forward(self,x):
        
        return self.rafire(x)
        
ec_net = EC_NET().type(torch.float32).to(device).eval()
ec_net= nn.DataParallel(ec_net).to(device)
#ec_net.load_state_dict(torch.load("/kaggle/input/encoder-weight-data/weight.pth",weights_only=False,map_location=torch.device('cpu')))

j=0
for i in x:
    print(f"loding batch : {j}")
    if j==0:
        encode = ec_net(i).cpu().detach().numpy().reshape((len(i),10))
    else:
        encode = numpy.concatenate((encode, ec_net(i).cpu().detach().numpy().reshape((len(i),10))), axis=0)
        
    #if j==25:
    #    break
    j+=1

x=torch.Tensor(encode)

loding batch : 0


In [5]:
y=pandas.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv").sort_values(by=['date', 'warehouse'])
y=torch.from_numpy(numpy.nan_to_num(y['sales'].to_numpy().astype(numpy.float32))).reshape(len(y),1)[::100]

In [6]:
x.shape, y.shape, x[0].shape

(torch.Size([308263, 10]), torch.Size([40075, 1]), torch.Size([10]))

In [7]:
train_x = torch.utils.data.DataLoader(x,batch_size=batch_size)
train_y = torch.utils.data.DataLoader(y,batch_size=batch_size)

In [8]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=0.02)
        
class EffnetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.rafire_1 = nn.LSTMCell(10, 1524)
                                     #nn.BatchNorm1d(1524),
                                     #nn.ReLU())

        self.rafire_2 = nn.LSTMCell(1524, 1324)
                                     #nn.BatchNorm1d(1324),
                                     #nn.LeakyReLU())

        self.rafire_3 = nn.LSTMCell(1324, 1024)
                                     #nn.BatchNorm1d(1024),
                                     #nn.LeakyReLU())

        self.rafire_4 = nn.LSTMCell(1024, 824)
                                     #nn.BatchNorm1d(824),
                                     #nn.LeakyReLU())

        self.rafire_5 = nn.LSTMCell(824, 524)
                                     #nn.BatchNorm1d(524),
                                     #nn.LeakyReLU())

        self.rafire_6 = nn.LSTMCell(524, 324)
                                     #nn.BatchNorm1d(324),
                                     #nn.LeakyReLU())

        self.rafire_7 = nn.LSTMCell(324, 124)
                                    #nn.BatchNorm1d(124),
                                    #nn.LeakyReLU())
            
        self.rafire_8 = nn.Linear(124, 1)

    def forward(self,x):
        x,_=self.rafire_1(x)
        x,_=self.rafire_2(x)
        x,_=self.rafire_3(x)
        x,_=self.rafire_4(x)
        x,_=self.rafire_5(x)
        x,_=self.rafire_6(x)
        x,_=self.rafire_7(x)
        
        return self.rafire_8(x)



In [9]:
EFF_NET = EffnetModel().float()
EFF_NET= nn.DataParallel(EFF_NET).to(device)

In [10]:
criterion = nn.L1Loss()

optimizer = optim.AdamW(EFF_NET.parameters(), lr=lr,betas=(beta1, 0.999))
scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.86)

In [ ]:
high_rf,i,j_r,k,z_w_=100000,0,0,0,[]
if os.path.exists(f"/kaggle/working/jsr_rf")==False:
    os.mkdir(f"/kaggle/working/jsr_rf")
    
correct,total=0,0

while(True):
    optimizer.zero_grad()
    for data,label in zip(train_x,train_y):
        #z_ = nn.Embedding(10, 3, dtype=torch.float64)
        #print(z_(torch.FloatTensor(data)))
        
        output = EFF_NET(torch.reshape(data,(len(data),10)).to(device)).view(-1)
        err_real = criterion(output, label.to(device))
        err_real.backward()
        optimizer.step()
        #print(err_real.item())
        
    print(f"EPOCH : {i} LOSS_wl : {err_real.item()}")

    if len(z_w_)>=3:
        if len([True for i in range(1,4) if z_w_[len(z_w_)-i]<=z_w_[len(z_w_)-3] and z_w_[len(z_w_)-i]>=z_w_[len(z_w_)-4]])==3:
            z_w_,j_r=[],0
            print(f"lr_br:{optimizer.param_groups[0]['lr']}".upper())
            scheduler.step()
            print(f"lr_up:{optimizer.param_groups[0]['lr']}".upper())
            
    z_w_.append(err_real.item())
    if err_real.item()<high_rf:
            high_rf=err_real.item()
            torch.save(EFF_NET.state_dict(),f'/kaggle/working/jsr_rf/{err_real.item()}.pth')
    if i==10:
        break
    i+=1

/usr/local/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


EPOCH : 0 LOSS_wl : 32.03329086303711
EPOCH : 1 LOSS_wl : 21.88775062561035
EPOCH : 2 LOSS_wl : 29.548120498657227
